## Assignment 3
### Name: Kshitij Mohan Kadam
### Roll NO: 43133
### Batch: Q9

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
print(tf.__version__)

2.10.0


### 1. Loading and Preprocessing the image data

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [3]:
print(f"train data shape: {x_train.shape}")
print(f"label train shape: {y_train.shape}")
print(f"test data shape: {x_test.shape}")
print(f"label test shape: {y_test.shape}")

train data shape: (50000, 32, 32, 3)
label train shape: (50000, 1)
test data shape: (10000, 32, 32, 3)
label test shape: (10000, 1)


In [4]:
y_train[0]

array([6], dtype=uint8)

In [5]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

In [6]:
print(f"label train shape: {y_train.shape}")
print(f"label test shape: {y_test.shape}")

label train shape: (50000, 10)
label test shape: (10000, 10)


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
def get_generator_aug():
    data_generator_aug = ImageDataGenerator(rescale=(1/255.0),
                                       rotation_range=35,
                                       width_shift_range=0.3,
                                       height_shift_range=0.3,
                                       fill_mode='nearest',
                                       brightness_range=(0.2,0.8),
                                       shear_range=45.0,
                                       horizontal_flip=True,
                                       vertical_flip=True,
                                       zoom_range=[0.5, 1.5])
    return data_generator_aug

In [9]:
data_generator_aug = get_generator_aug()
data_generator_aug.fit(x_train)
train_generator_aug = data_generator_aug.flow(x_train, y_train, batch_size=10, shuffle=True)

data_generator_aug_test = get_generator_aug()
data_generator_aug_test.fit(x_test)
train_generator_aug = data_generator_aug.flow(x_train, y_train, batch_size=10, shuffle=True)

In [10]:
data_generator = ImageDataGenerator(rescale=(1/255.0))
data_generator.fit(x_train)
img_generator = data_generator.flow(x_train, y_train, batch_size=10, shuffle=False)

### 2. Defining model architecture

In [11]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

In [12]:
def get_model(input_shape):
  input_layer = Input(input_shape)
  layer1 = Conv2D(32, 8, activation='relu', padding='SAME')(input_layer)
  layer2 = MaxPooling2D((2,2))(layer1)
  layer3 = Conv2D(32, 4, activation='relu', padding='SAME')(layer2)
  layer4 = MaxPooling2D((2,2))(layer3)
  layer5 = Flatten()(layer4)
  layer6 = Dense(16, activation="relu")(layer5)
  output_layer = Dense(10, activation='softmax')(layer6)

  model = Model(inputs=input_layer, outputs=output_layer)

  model.compile(optimizer=tf.keras.optimizers.Adam(3e-4),
                loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])
  return model

In [13]:
model = get_model((32,32,3))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        6176      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 32)        16416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 8, 8, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2048)              0     

### 3. Training the model

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

train_steps_per_epoch = img_generator.n // img_generator.batch_size

In [15]:
history = model.fit(img_generator, steps_per_epoch=train_steps_per_epoch, validation_data=(x_test,y_test),epochs=10, callbacks=[EarlyStopping(monitor='val_categorical_accuracy', patience=5)])

Epoch 1/10
5000/5000 [==============================] - 50s 10ms/step - loss: 1.7500 - categorical_accuracy: 0.3627 - val_loss: 158.7834 - val_categorical_accuracy: 0.4292
Epoch 2/10
5000/5000 [==============================] - 47s 9ms/step - loss: 1.3638 - categorical_accuracy: 0.5047 - val_loss: 210.6111 - val_categorical_accuracy: 0.4439
Epoch 3/10
5000/5000 [==============================] - 48s 10ms/step - loss: 1.2249 - categorical_accuracy: 0.5593 - val_loss: 181.1259 - val_categorical_accuracy: 0.4783
Epoch 4/10
5000/5000 [==============================] - 47s 9ms/step - loss: 1.1409 - categorical_accuracy: 0.5905 - val_loss: 176.2382 - val_categorical_accuracy: 0.5090
Epoch 5/10
5000/5000 [==============================] - 53s 11ms/step - loss: 1.0737 - categorical_accuracy: 0.6165 - val_loss: 179.3501 - val_categorical_accuracy: 0.5087
Epoch 6/10
5000/5000 [==============================] - 52s 10ms/step - loss: 1.0244 - categorical_accuracy: 0.6348 - val_loss: 188.1238 - val

### 4. Evaluating its performance

In [16]:
from sklearn.metrics import accuracy_score

In [17]:
y_pred = model.predict(x_test)

313/313 [==============================] - 2s 5ms/step


In [18]:
y_pred = np.argmax(y_pred, axis=1)

In [19]:
y_test = np.argmax(y_test, axis=1)

In [20]:
print(accuracy_score(y_test, y_pred))

0.5134
